# DIY Interactive Segmentation with napari

This is Kyle Harrington's tutorial adapted from [here](https://github.com/kephale/napari-dl-at-mbl-2024/blob/main/napari-workshops/notebooks/diy_interactive_segmentation.ipynb) adapted for the SEM grain images shared [here](https://forum.image.sc/t/free-article-from-machine-learning-to-deep-learning-revolutionizing-microscopy-image-analysis/106340/3)

BN:  I added in live instance segmentation so we can see if the objects in the image are separated. 

+++

napari is a very flexible and "hackable" tool. In this tutorial we will
make a custom interactive segmentation tool from scratch.

+++


In [ ]:
from appdirs import user_data_dir
import os
import zarr
import dask.array as da
import toolz as tz

from sklearn.ensemble import RandomForestClassifier

from skimage import data, segmentation, feature, future
from skimage.feature import multiscale_basic_features
from skimage.io import imread, imshow
import numpy as np
from functools import partial
import napari
import threading

from ome_zarr.io import parse_url
from ome_zarr.reader import Reader

from functools import partial
from psygnal import debounced
from superqt import ensure_main_thread

import logging
import sys

from skimage.measure import label

LOGGER = logging.getLogger("halfway_to_i2k_2023_america")
LOGGER.setLevel(logging.DEBUG)

streamHandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
streamHandler.setFormatter(formatter)
LOGGER.addHandler(streamHandler)

# Open the data

For simplicity we will just use the first channel

In [ ]:
from pathlib import Path

#tnia_images_path = Path(r"D:/images/tnia-python-images")
tnia_images_path = Path(r"/home/bnorthan/images/tnia-python-images/")

parent_path = tnia_images_path / "imagesc/2024_12_19_sem_grain_size_revisit"
image_name = "211122_AM_Al2O3_SE_021"
image = imread(str(parent_path / ("211122_AM_Al2O3_SE_021.ome.tiff")))

prediction_name = image_name+"_prediction_cells"
painting_name = image_name+"painting_cells"
instance_name = image_name+"instance_cells"
instance_painting_name = image_name+"instance_painting_cells"


### Visualize in Napari

In [ ]:
viewer = napari.Viewer()

data_layer = viewer.add_image(image)
data_layer.bounding_box.visible = True

### Extract Features

In [ ]:
def extract_features(image, feature_params):
    features_func = partial(
        multiscale_basic_features,
        intensity=feature_params["intensity"],
        edges=feature_params["edges"],
        texture=feature_params["texture"],
        sigma_min=feature_params["sigma_min"],
        sigma_max=feature_params["sigma_max"],
        channel_axis=None,
    )
    # print(f"image shape {image.shape} feature params {feature_params}")

    if len(image.shape) == 2:
        features = features_func(image)
        return features
    elif len(image.shape) == 3:
        
        for c in range(image.shape[-1]):
            features_temp = features_func(np.squeeze(image[..., c]))
            if c == 0:
                features = features_temp
            else:
                features = np.concatenate((features, features_temp), axis=2)
        return features

example_feature_params = {
    "sigma_min": 1,
    "sigma_max": 5,
    "intensity": True,
    "edges": True,
    "texture": True,
}


features = extract_features(image, example_feature_params)
features.shape

### Visualize Features

In [ ]:
def show_features():
    for feature_idx in range(features.shape[-1]):
        viewer.add_image(features[..., feature_idx])
        
#show_features()

### Making the Interactive Segmentation Tool!

Ok, now we've seen:
- our data
- some features we can compute for our data

Our goal is to create an image where we have labels that correspond to the zebrafish sample. 

The approach is that when we annotate/draw in our painting layer, then we want our segmentations to be updated automatically. 

We will do this using 3 different image layers:

1. Our input image
2. A layer for painting
3. A layer for storing the machine learning generated predictions

Due to popular demand we will be using Zarr to store these layers, because that will help this approach scale to very large datasets. However, we could have used numpy arrays as well.

+++

### Create our painting and prediction layers


In [ ]:
zarr_path = r"./zarr"
print(f"Saving outputs to zarr path: {zarr_path}")

# Create a prediction layer
prediction_data = zarr.open(
    f"{zarr_path}/{prediction_name}",
    mode='a',
    shape=image.shape[:2],
    dtype='i4',
    dimension_separator="/",

)
prediction_layer = viewer.add_labels(prediction_data, name="Prediction", scale=data_layer.scale)

# Create a painting layer
painting_data = zarr.open(
    f"{zarr_path}/{painting_name}",
    mode='a',
    shape=image.shape[:2],
    dtype='i4',
    dimension_separator="/",
)

painting_layer = viewer.add_labels(painting_data, name="Painting", scale=data_layer.scale)

# create a instance layer for painting
instance_layer = zarr.open(
    f"{zarr_path}/{instance_name}",
    mode='a',
    shape=image.shape[:2],
    dtype='i4',
    dimension_separator="/",
)

instance_layer = viewer.add_labels(instance_layer, name="Instance", scale=data_layer.scale)

instance_painting_data = zarr.open(
    f"{zarr_path}/{instance_painting_name}",
    mode='a',
    shape=image.shape[:2],
    dtype='i4',
    dimension_separator="/",
)

instance_painting_layer = viewer.add_labels(instance_painting_data, name="Instance Painting", scale=data_layer.scale)

### Let's make a UI as well

In [ ]:
from qtpy.QtWidgets import (
    QVBoxLayout,
    QHBoxLayout,
    QComboBox,
    QLabel,
    QCheckBox,
    QDoubleSpinBox,
    QGroupBox,
    QWidget,
)

class NapariMLWidget(QWidget):
    def __init__(self, parent=None):
        super(NapariMLWidget, self).__init__(parent)

        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        # Dropdown for selecting the model
        model_label = QLabel("Select Model")
        self.model_dropdown = QComboBox()
        self.model_dropdown.addItems(["Random Forest"])
        model_layout = QHBoxLayout()
        model_layout.addWidget(model_label)
        model_layout.addWidget(self.model_dropdown)
        layout.addLayout(model_layout)

        # Select the range of sigma sizes
        self.sigma_start_spinbox = QDoubleSpinBox()
        self.sigma_start_spinbox.setRange(0, 256)
        self.sigma_start_spinbox.setValue(1)

        self.sigma_end_spinbox = QDoubleSpinBox()
        self.sigma_end_spinbox.setRange(0, 256)
        self.sigma_end_spinbox.setValue(5)

        sigma_layout = QHBoxLayout()
        sigma_layout.addWidget(QLabel("Sigma Range: From"))
        sigma_layout.addWidget(self.sigma_start_spinbox)
        sigma_layout.addWidget(QLabel("To"))
        sigma_layout.addWidget(self.sigma_end_spinbox)
        layout.addLayout(sigma_layout)

        # Boolean options for features
        self.intensity_checkbox = QCheckBox("Intensity")
        self.intensity_checkbox.setChecked(True)
        self.edges_checkbox = QCheckBox("Edges")
        self.texture_checkbox = QCheckBox("Texture")
        self.texture_checkbox.setChecked(True)

        features_group = QGroupBox("Features")
        features_layout = QVBoxLayout()
        features_layout.addWidget(self.intensity_checkbox)
        features_layout.addWidget(self.edges_checkbox)
        features_layout.addWidget(self.texture_checkbox)
        features_group.setLayout(features_layout)
        layout.addWidget(features_group)

        # Dropdown for data selection
        data_label = QLabel("Select Data for Model Fitting")
        self.data_dropdown = QComboBox()
        self.data_dropdown.addItems(
            ["Current Displayed Region", "Whole Image"]
        )
        self.data_dropdown.setCurrentText("Current Displayed Region")
        data_layout = QHBoxLayout()
        data_layout.addWidget(data_label)
        data_layout.addWidget(self.data_dropdown)
        layout.addLayout(data_layout)

        # Checkbox for live model fitting
        self.live_fit_checkbox = QCheckBox("Live Model Fitting")
        self.live_fit_checkbox.setChecked(True)
        layout.addWidget(self.live_fit_checkbox)

        # Checkbox for live prediction
        self.live_pred_checkbox = QCheckBox("Live Prediction")
        self.live_pred_checkbox.setChecked(True)
        layout.addWidget(self.live_pred_checkbox)

        # Checkbox for live instance segmentation
        self.live_instance_checkbox = QCheckBox("Live Instance Segmentation")
        self.live_instance_checkbox.setChecked(True)
        layout.addWidget(self.live_instance_checkbox)
        
        self.setLayout(layout)
        

In [ ]:
# Let's add this widget to napari

widget = NapariMLWidget()
viewer.window.add_dock_widget(widget, name="halfway to I2K 2023 America")

### We have a widget, we have our painting and prediction layers, now what?

We need to start connecting things together. How should we do that? napari has things called "events" that happen when things happen within napari. We want to respond to a few different event types:
- changes in camera (e.g. camera position and rotation)
- changes in "dims" (e.g. moving a dimension slider)
- painting events (e.g. a user clicked, painted, and release their mouse)

When one of these events happens, we want to: 
- update our machine learning model with the new painted data
- update our prediction with the updated ML model


In [ ]:
# Let's start with our event listener

# We use "curry" because this allows us to "store" our viewer and widget for later use
@tz.curry
def on_data_change(event, viewer=None, widget=None):
    corner_pixels = data_layer.corner_pixels

    # Ensure the painting layer visual is updated
    painting_layer.refresh()

    # Training the ML model and generating predictions can take time
    #   we will use a "thread" to perform these calculations
    #   otherwise napari will freeze until these
    # calculations are done
    thread = threading.Thread(
        target=threaded_on_data_change,
        args=(
            event,
            corner_pixels,
            viewer.dims,
            widget.model_dropdown.currentText(),
            {
                "sigma_min": widget.sigma_start_spinbox.value(),
                "sigma_max": widget.sigma_end_spinbox.value(),
                "intensity": widget.intensity_checkbox.isChecked(),
                "edges": widget.edges_checkbox.isChecked(),
                "texture": widget.texture_checkbox.isChecked(),
            },
            widget.live_fit_checkbox.isChecked(),
            widget.live_pred_checkbox.isChecked(),
            widget.live_instance_checkbox.isChecked(),
            widget.data_dropdown.currentText(),
        ),
    )
    thread.start()
    thread.join()

    # Ensure the prediction layer visual is updated
    prediction_layer.refresh()
    instance_layer.refresh()

In [ ]:
# Now we have to make the hard part of the listener

model = None

def threaded_on_data_change(
    event,
    corner_pixels,
    dims,
    model_type,
    feature_params,
    live_fit,
    live_prediction,
    live_instance, 
    data_choice,
):
    global model
    LOGGER.info(f"Labels data has changed! {event}")

    current_step = dims.current_step

    LOGGER.info("make mask")
    # Find a mask of indices we will use for fetching our data
    mask_idx = (slice(corner_pixels[0, 0], corner_pixels[1, 0]), slice(corner_pixels[0, 1], corner_pixels[1, 1]))
    #if data_choice == "Whole Image":
    #    mask_idx = tuple([slice(0, sz) for sz in data_layer.data.shape])

    LOGGER.info(f"mask idx {mask_idx}, image {data_layer.data.shape}")
    active_image = data_layer.data[mask_idx]
    LOGGER.info(
        f"active image shape {active_image.shape} data choice {data_choice} painting_data {painting_data.shape} mask_idx {mask_idx}"
    )

    active_labels = painting_data[mask_idx]

    def compute_features(image, feature_params):
        """Compute features for each channel and concatenate them."""
        features = extract_features(
            image, feature_params
        )

        return features

    training_labels = None

    if data_choice == "Current Displayed Region":
        # Use only the currently displayed region.
        training_features = compute_features(
            active_image, feature_params
        )
        training_labels = np.squeeze(active_labels)

    else:
        raise ValueError(f"Invalid data choice: {data_choice}")

    if (training_labels is None) or np.any(training_labels.shape == 0):
        LOGGER.info("No training data yet. Skipping model update")
    elif live_fit:
        # Retrain model
        LOGGER.info(
            f"training model with labels {training_labels.shape} features {training_features.shape} unique labels {np.unique(training_labels[:])}"
        )
        model = update_model(training_labels, training_features, model_type)

    # Don't do live prediction on whole image, that happens earlier slicewise
    if live_prediction:
        # Update prediction_data
        prediction_features = compute_features(
            active_image, feature_params
        )
        # Add 1 becasue of the background label adjustment for the model
        prediction = predict(model, prediction_features, model_type)
        LOGGER.info(
            f"prediction {prediction.shape} prediction layer {prediction_layer.data.shape} prediction {np.transpose(prediction).shape} features {prediction_features.shape}"
        )

        #if data_choice == "Whole Image":
        prediction_layer.data[mask_idx] = np.transpose(prediction)

    if live_instance:
        temp = prediction_layer.data[mask_idx].copy()
        mask = active_labels > 0
        #print('active labels ',active_labels.shape, type(active_labels) )
        #print('the mask type is: !!!!', mask.shape, type(mask))
        temp = np.where(mask, active_labels, temp)
        mask = instance_painting_layer.data[mask_idx] > 0
        temp = np.where(mask, instance_painting_layer.data[mask_idx], temp) 
        temp = temp-1
        labels = label(temp)
        instance_layer.data[mask_idx] = labels
        #else:
        #    prediction_layer.data[mask_idx] = np.transpose(prediction)[
        #        np.newaxis, :
        #    ]

In [ ]:
# Model training function that respects widget's model choice
def update_model(labels, features, model_type):
    features = features[labels > 0, :]
    # We shift labels - 1 because background is 0 and has special meaning, but models need to start at 0
    labels = labels[labels > 0] - 1
    
    if model_type == "Random Forest":
        clf = RandomForestClassifier(
            n_estimators=50, n_jobs=-1, max_depth=10, max_samples=0.05
        )

    print(
        f"updating model with label shape  {labels.shape} feature shape {features.shape} unique labels {np.unique(labels)}"
    )
    
    clf.fit(features, labels)

    return clf


def predict(model, features, model_type):
    # We shift labels + 1 because background is 0 and has special meaning
    prediction = future.predict_segmenter(features.reshape(-1, features.shape[-1]), model).reshape(features.shape[:-1]) + 1

    return np.transpose(prediction)

In [ ]:
# Now connect everything together
for listener in [
    viewer.camera.events,
    viewer.dims.events,
    painting_layer.events.paint,
    instance_painting_layer.events.paint,
]:
    listener.connect(
        debounced(
            ensure_main_thread(
                on_data_change(
                    viewer=viewer,
                    widget=widget,  # pass the widget instance for easy access to settings
                )
            ),
            timeout=1000,
        )
    )

## Fill Holes Save ML pixel classificaton result 

In [ ]:
import numpy as np
from scipy.ndimage import binary_fill_holes
from skimage.measure import label
from scipy.ndimage import find_objects

result = instance_layer.data[:]

labels = label(result)

# Create a filled labels array
filled_labels = np.zeros_like(labels)

# Get slices for each label (bounding boxes)
slices = find_objects(labels)

# Iterate over each label's bounding box
for label_value, label_slice in enumerate(slices, start=1):
    if label_slice is None:  # Skip missing labels
        continue

    # Crop to the bounding box
    cropped_labels = labels[label_slice]

    # Create a binary mask for the current label
    binary_mask = cropped_labels == label_value

    # Fill holes in the binary mask
    filled_mask = binary_fill_holes(binary_mask)

    # Assign filled mask back to the output array within the cropped region
    filled_labels[label_slice][filled_mask] = label_value

# Add to viewer
viewer.add_labels(filled_labels, name="Filled Labels")

In [ ]:
filled_labels.dtype

In [ ]:
from skimage.io import imsave

# print layers of viewer
print(viewer.layers)

# get prediction layer
result = viewer.layers[1].data[:]
result = viewer.layers['Instance'].data[:]  

import matplotlib.pyplot as plt

plt.imshow(result)

imsave(parent_path / (image_name+"_segmented.tif"), result)
imsave(parent_path / ("ml_hole_fill_segmented.png"), filled_labels.astype('uint16'))

In [ ]:

import matplotlib.pyplot as plt

plt.imshow(result)

imsave(parent_path / (image_name+"_segmented.tif"), result)
imsave(parent_path / ("ml_hole_fill_segmented.png"), filled_labels)